In [1]:
from bs4 import BeautifulSoup
import requests
import urllib.request
import urllib.parse
import redis
import re
import json


In [24]:
prog_name = "kyobobookparsor"

def readCacheOrUpdate( page_num ):

    prefix = prog_name + ':ebook:category:it:page:'
    key = prefix + str(page_num)

    cache = rd.get(key)

    if cache is None:
        req = requests.get('https://autoever.dkyobobook.co.kr/content/contentList.ink?brcd=&sntnAuthCode=&contentAll=Y&cttsDvsnCode=001&ctgrId=24&orderByKey=publDate&selViewCnt=20&recordCount=20&pageIndex='+ str(page_num) , verify=False)

        html = req.text
        rd.set(key, html)

        return html
    else:
        return rd.get(key)
    

In [3]:
rd = redis.StrictRedis(host='localhost', port=6379, db=0, charset="utf-8", decode_responses=True)


In [57]:
def parseAndStoreContent( html ):
    
    bs = BeautifulSoup(html)
    titles = bs.select('ul.book_resultList li ul li.tit a')
    texts = bs.select('ul.book_resultList li ul li.txt')
    images = bs.select('ul.book_resultList li div.img img')
       
    print(len(titles))

    for item_in_page in range(len(titles)):
        title = titles[item_in_page].string
        
        # print(title)

        text = texts[item_in_page].string
        id = str(images[item_in_page]).split('/')[-3]
        imgpath = 'https:' + images[item_in_page]['src']

        key = prog_name + ':ebook:id:' + id

        # print({"title" : title, "desc" : text, "imgpath" : imgpath})
        # print(json.dumps({"title" : title, "desc" : text, "imgpath" : imgpath}, ensure_ascii=False))

        rd.set(key, json.dumps({"title" : title, "desc" : text, "imgpath" : imgpath}, ensure_ascii=False))




In [65]:
# parseAndStoreContent(readCacheOrUpdate(1))
# rd.keys(prog_name + ":ebook:category:it:*")
# rd.keys(prog_name + ":ebook:id:*")
content = json.loads(rd.get('kyobobookparsor:ebook:id:480D210307680'))
content['title']

'[무료 특별판] 3분 엑셀'